In [1]:
import datetime
import QuantExt as qe # 퀀트립 확장판 불러오기
from module.FX_Curve import GET_QUOTE, USDIRS_CURVE, KRWCCS_CURVE # 이전에 만든 함수들 가져오기(여기선 Fx_curve)

In [2]:
class CCS():
    def __init__(self, todays_date, effective_date, maturity_date, ccs_rate, fx_spot, usd_notional, position):
        
        # Initial Setup 1 - Date / Curves / FX Spot
        self.date = todays_date # 오늘 날짜
        self.usd_curve = self.USD_CURVE(self.date) # 달러 IRS 커브
        self.krw_curve = self.KRW_CURVE(self.date) # 원화 CCS 커브
        self.fx_spot = fx_spot # 환율
        
        # Initial Setup 2 - Instrument Info(시장 정보)
        self.effective_date = qe.Date(effective_date.day, effective_date.month, effective_date.year) # 퀀트립 형태의 날짜로 바꿈
        self.maturity_date = qe.Date(maturity_date.day, maturity_date.month, maturity_date.year) # 퀀트립 형태의 날짜로 바꿈
        
        if position == 'Long': # 포지션 방향(매수 or 매도)
            self.position = qe.VanillaSwap.Payer # 원화 고정 금리를 내고 달러 변동 금리를 받음
        else:
            self.position = qe.VanillaSwap.Receive # 원화 고정 금리를 받고 달러 변동 금리를 냄
        
        self.ccs_rate = ccs_rate # 통화스왑 금리
        self.usd = qe.USDCurrency()
        self.usd_notional = usd_notional # 달러 명목 금액
        self.krw = qe.KRWCurrency()
        self.krw_notional = usd_notional * fx_spot # 원화 명목 금액은 계산함

        self.length = 2 # 거래일과 효력 발생일의 간격 일수
        self.spread = 0.0 # 일반적으로 0
        self.convention = qe.ModifiedFollowing # 휴일 처리 방식
        self.calendar = qe.JointCalendar(qe.SouthKorea(), qe.UnitedStates()) # 달력(한국 달력, 미국 달력)
        self.tenor = qe.Period(6, qe.Months) # 이자 지급 주기(통화스왑의 경우 6개월)
        
        self.fixed_day_count = qe.Actual365Fixed() # 날짜 계산(원화의 경우)
        self.float_day_count = qe.Actual360() # 날짜 계산(달러의 경우)
        self.dateGeneration = qe.DateGeneration.Backward # 이자 지급 간격 방식(Backward -> 뒤에서부터 간격 계산)
        
        # Pricing Results
        self.npv = self.PRICING(self.usd_curve, self.krw_curve, self.fx_spot) # 환율스왑 이론가 계산 함수
        self.fx_delta = self.FX_DELTA() # 환율이 1% 움직였을 때 환율스왑의 환델타 계산
        self.usd_curve_delta = self.USD_CURVE_DELTA() # 달러 금리가 1bp 움직였을 때 통화스왑의 달러 금리 델타 계산
        self.krw_curve_delta = self.KRW_CURVE_DELTA() # 원화 금리가 1bp 움직였을 때 통화스왑의 원화 금리 델타 계산
        self.theta = self.THETA() # 날짜가 하루 지났을 때 환율스왑의 세타 계산
        
    def USD_CURVE(self, date): # Fx_curve에서 구현한 USD_CURVE(IRS) 불러오는 함수
        return USDIRS_CURVE(date, GET_QUOTE(date, 'USD'))
    
    def KRW_CURVE(self, date): # Fx_curve에서 구현한 KRW_CURVE(CCS) 불러오는 함수
        return KRWCCS_CURVE(date, GET_QUOTE(date, 'KRW'))
    
    def PRICING(self, usd_curve, krw_curve, fx_spot): # 실제 통화스왑의 이론가를 계산하는 기능을 담당하는 함수 
        # handles
        usd_curve_handle = qe.YieldTermStructureHandle(usd_curve)
        krw_curve_handle = qe.YieldTermStructureHandle(krw_curve)
        fx_spot_handle = qe.QuoteHandle(qe.SimpleQuote(fx_spot))
        
        usd_6m_libor = qe.USDLibor(qe.Period(6, qe.Months), usd_curve_handle) # 이자 지급 주기(6개월, 기준)
        usd_6m_libor.addFixing(qe.Date(25, 11, 2020), 0.0009) # 25. 11. 20 날짜의 금리? 직접 기입해야 함(없으면 계산이 안됨)
        
        # 고정 금리 스케줄(효력 발생일, 만기일, 이자 지급 주기, 달력, 중간의 이자 지급 방식, 만기일자일 때의 이자 지급 방식, 날짜 생성)
        fixed_schedule = qe.Schedule(self.effective_date,
                                     self.maturity_date,
                                     self.tenor,
                                     self.calendar,
                                     self.convention,
                                     self.convention,
                                     self.dateGeneration,
                                     False) # False이므로 월말 기준
        
        # 변동 금리 스케줄(효력 발생일, 만기일, 이자 지급 주기, 달력, 중간의 이자 지급 방식, 만기일자일 때의 이자 지급 방식, 날짜 생성)
        float_schedule = qe.Schedule(self.effective_date,
                                     self.maturity_date,
                                     self.tenor,
                                     self.calendar,
                                     self.convention,
                                     self.convention,
                                     self.dateGeneration,
                                     False)
        
        # CCS(퀀트립 확장판에만 존재)
        ccs = qe.CrossCcyFixFloatSwap(self.position, # 포지션(매수 or 매도)
                                      self.krw_notional, # 원화 명목 금액
                                      self.krw, # 원화
                                      fixed_schedule, # 고정 금리 스케줄
                                      self.ccs_rate, # 통화스왑 금리
                                      self.fixed_day_count, # 고정 금리 날짜 카운트
                                      self.convention, # 이자 지금 방식
                                      self.length, # 거래일 ~ 효력 발생일 간격(2일로 설정되어 있음)
                                      self.calendar, # 달력
                                      self.usd_notional, # 달러 명목 금액
                                      self.usd, # 달러
                                      float_schedule, # 변동 금리 스케줄
                                      usd_6m_libor, # 이자 지급 주기(6개월로 설정되어 있음)
                                      self.spread, # 0으로 설정되어 있음
                                      self.convention, # 이자 지급 방식
                                      self.length, # 거래일 ~ 효력 발생일 간격(2일로 설정되어 있음)
                                      self.calendar) # 달력
        
        # engine
        engine = qe.CrossCcySwapEngine(self.krw, # 원화
                                       krw_curve_handle, # 원화 CCS 커브
                                       self.usd, # 달러
                                       usd_curve_handle, # 달러 IRS 커브
                                       fx_spot_handle) # 환율
        
        ccs.setPricingEngine(engine)
        npv = ccs.NPV()
        return npv
        
    def FX_DELTA(self):
        # 1%
        percentage = 0.01 # 환율 1%를 의미
        
        up_fx = self.fx_spot * (1 + percentage)
        up_fxf = self.PRICING(self.usd_curve, self.krw_curve, up_fx)
        
        down_fx = self.fx_spot * (1 - percentage)
        down_fxf = self.PRICING(self.usd_curve, self.krw_curve, down_fx)
        
        fx_delta = (up_fxf - down_fxf) / 2
        return fx_delta
    
    def USD_CURVE_DELTA(self): # 선도환의 달러 금리 델타
        curve_handle = qe.YieldTermStructureHandle(self.usd_curve)
        
        # 1bp
        basis_point = 0.0001 # 0.01%를 의미
        
        # FRA price when 1bp up
        up_curve = qe.ZeroSpreadedTermStructure(curve_handle, qe.QuoteHandle(qe.SimpleQuote(basis_point)))
        up_fxf = self.PRICING(up_curve, self.krw_curve, self.fx_spot)
        
        # FRA price when 1bp down
        down_curve = qe.ZeroSpreadedTermStructure(curve_handle, qe.QuoteHandle(qe.SimpleQuote(-basis_point)))
        down_fxf = self.PRICING(down_curve, self.krw_curve, self.fx_spot)

        # USD Curve Delta
        delta = (up_fxf - down_fxf) / 2
        return delta
    
    def KRW_CURVE_DELTA(self): # 선도환의 원화 금리 델타
        curve_handle = qe.YieldTermStructureHandle(self.krw_curve)
        
        # 1bp
        basis_point = 0.0001 # 0.01%를 의미
        
        # FRA price when 1bp up
        up_curve = qe.ZeroSpreadedTermStructure(curve_handle, qe.QuoteHandle(qe.SimpleQuote(basis_point)))
        up_fxf = self.PRICING(self.usd_curve, up_curve, self.fx_spot)
        
        # FRA price when 1bp down
        down_curve = qe.ZeroSpreadedTermStructure(curve_handle, qe.QuoteHandle(qe.SimpleQuote(-basis_point)))
        down_fxf = self.PRICING(self.usd_curve, down_curve, self.fx_spot)

        # KRW Curve Delta
        delta = (up_fxf - down_fxf) / 2
        return delta
    
    def THETA(self):
        price_t0 = self.PRICING(self.usd_curve, self.krw_curve, self.fx_spot)
        usd_curve_t1 = self.USD_CURVE(self.date + datetime.timedelta(days=1))
        krw_curve_t1 = self.KRW_CURVE(self.date + datetime.timedelta(days=1))
        price_t1 = self.PRICING(usd_curve_t1, krw_curve_t1, self.fx_spot)
        
        theta = price_t1 - price_t0
        return theta

In [3]:
if __name__ == "__main__":
    
    # Today's Date
    todays_date = datetime.date(2020, 10, 8)
    
    # FX Spot Rate
    fx_spot = 1133.85 # 시장 환율

    # CCS Instrument Info
    effective_date = datetime.date(2020, 11, 26) # 효력 발생 일
    maturity_date = datetime.date(2025, 11, 26) # 만기일
    position = 'Long' # 포지셔닝
    ccs_rate = 0.0009746 # 통화스왑 금리
    usd_notional = 10000000 # 달러 명목 금액(천만 불)
    
    # Build CCS Object
    ccs = CCS(todays_date, # 오늘 날짜
              effective_date, # 효력 발생 일
              maturity_date, # 만기일
              ccs_rate, # 통화스왑 금리
              fx_spot, # 환율
              usd_notional, # 달러 명목 금액
              position) # 포지셔닝
    
    print("Price = {}".format(round(ccs.npv, 4)))
    print("FX Delta = {}".format(round(ccs.fx_delta, 4)))
    print("USD Curve Delta = {}".format(round(ccs.usd_curve_delta, 4)))
    print("KRW Curve Delta = {}".format(round(ccs.krw_curve_delta, 4)))
    print("Theta = {}".format(round(ccs.theta, 4)))

Price = 64597787.7045
FX Delta = -98913.1778
USD Curve Delta = -581421.776
KRW Curve Delta = 5631694.2323
Theta = 18752.2128
